The first step for this project (and almost every project) is pull the data and validating it. Here, no sql queries are nessescary as the csv files have been provided. So the next step is validating and cleaning our data sets.

There's two data sets to validate, weekly_returns.csv and ticker_attributes.csv. 

Let's start by loading them into dataframes and taking a quick look via head and tail.

In [4]:
import pandas as pd
import numpy as np

weeklyreturnsdf = pd.read_csv('DataSets/weekly_returns.csv')
tickerattridf = pd.read_csv('DataSets/ticker_attributes.csv')

print(weeklyreturnsdf.info())
print("\n")
print(tickerattridf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           3649 non-null   object 
 1   ticker_id      3649 non-null   int64  
 2   weekly_return  3649 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 85.6+ KB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ticker_id    7 non-null      int64 
 1   ticker       7 non-null      object
 2   description  7 non-null      object
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes
None


The data set for ticker attributes is small enough to validate completely by just printing it. 

There's no issues with blank or missing data that we can see from checking Non-Null Count. The column ticker_id matches data type in both datasets so we'll be using that as our foreign key whenever we join the sets for summary. 

Date is Dtype object by default not datetime or datetime64[ns] which isn't an issue right now, but will need to be addressed in future loads of the csv (including  parse_dates=['date'] in future read_csv's)

Next we'll look at a sample of the data, min and max based on key fields and check for duplicates.


In [5]:
print(weeklyreturnsdf.head())
print("\n")
print(tickerattridf)

        date  ticker_id  weekly_return
0   6/3/2011          0       0.251305
1  6/10/2011          0      -2.924585
2  6/17/2011          0      -1.463312
3  6/24/2011          0       0.373256
4   7/1/2011          0       5.846155


   ticker_id ticker                                  description
0          0    EEM            iShares MSCI Emerging Markets ETF
1          1    EFA                        iShares MSCI EAFE ETF
2          2    IJR               iShares Core S&P Small Cap ETF
3          3    SPY                       SPDR S&P 500 ETF Trust
4          4    AGG         iShares Core U.S. Aggregate Bond ETF
5          5    JNK  SPDR Bloomberg Barclays High Yield Bond ETF
6          6    SHY           iShares 1-3 Year Treasury Bond ETF


In [6]:
print(weeklyreturnsdf[["ticker_id","weekly_return"]].describe())

         ticker_id  weekly_return
count  3649.000000    3649.000000
mean      3.002466       0.125429
std       1.998559       2.759498
min       0.000000    -100.000000
25%       1.000000      -0.420284
50%       3.000000       0.073433
75%       5.000000       0.812018
max       6.000000      44.000000


So, we do have to handle the issue of when the weekly return is -100%. How often do these extremes occur and are they valid data points?


In [14]:
sortedwrdf = weeklyreturnsdf.sort_values(by=['weekly_return'], ascending=True)
print(sortedwrdf.head())

           date  ticker_id  weekly_return
2502  5/10/2019          4    -100.000000
1502  3/13/2020          2     -16.636300
1503  3/20/2020          2     -16.308004
2025  3/20/2020          3     -14.523411
980   3/13/2020          1     -14.315790


looking at finance.yahoo.com we can get a much more reasonable value for ticket_id 4, AGG. While not exact, it does line up closely (within 3 decimal points).

| Date    |  Adjusted Close    | Weekly Return |
| ------- | ------------------ | ------------- |
| 4/26/19 |           103.1823 | 0.480623      |
| 5/3/19  |           103.0779 | -0.101159     |
| 5/10/19 |           103.3919 | 0.304620      |
| 5/17/19 |           103.7344 | 0.331262      |
| 5/24/19 |           104.0959 | 0.348512      |

compared to the provided dataset

| date    | ticker id  | weekly return  |
| ------- | ---------- | -------------- |
| 4/26/19 | 4          | 0.480628       |
| 5/3/19  | 4          | -0.1008987     |
| 5/10/19 | 4          | -100           |
| 5/17/19 | 4          | 0.33128262     |
| 5/24/19 | 4          | 0.3485322      |

So in Data Cleaning We'll replace -100 with 0.304620.

We still need to check for duplicates, missing dates, or date misalignment

In [16]:
duplicateRowsDF =weeklyreturnsdf[weeklyreturnsdf.duplicated(subset=['date','ticker_id'])]
print(duplicateRowsDF)

Empty DataFrame
Columns: [date, ticker_id, weekly_return]
Index: []


In [18]:
weeklyreturnsdf['date'] =  pd.to_datetime(weeklyreturnsdf['date'], format='%m/%d/%Y')
tickerdatesdf = weeklyreturnsdf.groupby(['ticker_id'])
tickerdatesdf2 = tickerdatesdf.agg(Minimum_Date=('date', np.min), Maximum_Date=('date', np.max))
print(tickerdatesdf2)

          Minimum_Date Maximum_Date
ticker_id                          
0           2011-06-03   2021-04-30
1           2011-05-06   2021-04-30
2           2011-05-06   2021-04-30
3           2011-05-06   2021-04-30
4           2011-05-06   2021-04-30
5           2011-05-06   2021-04-30
6           2011-05-06   2021-04-23


In [21]:
tickerdatesdf3 = tickerdatesdf.agg('count')
print(tickerdatesdf3)

           date  weekly_return
ticker_id                     
0           518            518
1           522            522
2           522            522
3           522            522
4           522            522
5           522            522
6           521            521


These counts match the missing date information. With the same apporoach we took to fixing the -100 weekly return we'll take to adding in the missing information.

for EEM

| Date    | Adj Close | Weekly Returns |
| ------- | --------- | -------------- |
| 4/29/11 | 40.355591 |                |
| 5/6/11  | 38.943138 | -3.500018      |
| 5/13/11 | 37.869686 | -2.756460      |
| 5/20/11 | 37.990761 | 0.319715       |
| 5/27/11 | 38.539589 | 1.444635       |
| 6/3/11  | 38.636452 | 0.251334       |

and the 6/3/11 calculated weekly return approxiamtely matches our data set

| date   | ticker_id  | weekly_return  |
| ------ | ---------- | -------------- |
| 6/3/11 | 0          | 0.2513051      |

Now we just need to get our missing week for SHY

| Date    | Adj Close | Weekly Returns |
| ------- | --------- | -------------- |
| 4/23/21 | 86.211014 | 0.000000%      |
| 4/30/21 | 86.231003 | 0.023186%      |

and the 4/23/21 calculated weekly return approxiamtely matches our data set

| date    | ticker_id  | weekly_return  |
| ------- | ---------- | -------------- |
| 4/23/21 | 6          | 0              |


For our next step we'll implement these changes in the data cleaning steps